In [45]:
import torch
import torchaudio
from speechbrain.pretrained import SpectralMaskEnhancement
import soundfile as sf
from tqdm import tqdm

%reload_ext autoreload
%autoreload 2
%matplotlib inline

import IPython

In [2]:
enhance_model = SpectralMaskEnhancement.from_hparams(
    source="speechbrain/metricgan-plus-voicebank",
)
# tmpdir = 'pretrained_models/metricgan-plus-voicebank'
# enhancer = SpectralMaskEnhancement.from_hparams(
#         source="speechbrain/mtl-mimic-voicebank",
#         savedir=tmpdir,
# )

In [3]:
noisy = enhance_model.load_audio("/home/yangwenhao/storage/dataset/CN-Celeb/eval/test/id00800-singing-01-001.wav").unsqueeze(0)

# Add relative length tensor
enhanced = enhance_model.enhance_batch(noisy, lengths=torch.tensor([1.]))

# Saving enhanced signal on disk
torchaudio.save('enhanced.wav', enhanced.cpu(), 16000)

In [9]:
orignial = sf.read('/home/yangwenhao/storage/dataset/CN-Celeb/eval/test/id00800-singing-01-001.wav')
enhanced = sf.read('enhanced.wav')

IPython.display.Audio(orignial[0], rate=16000)

In [11]:
IPython.display.Audio(enhanced[0], rate=16000)

In [ ]:
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(20, 10))
im1 = ax1.imshow(20 * np.log10(np.abs(Y[ :, 0, :400])), origin='lower')
ax1.set_xlabel('frames')
_ = ax1.set_title('orignal')
im2 = ax2.imshow(20 * np.log10(np.abs(Y_reduced_noise[:, 0, :400])), origin='lower')
ax2.set_xlabel('frames')
_ = ax2.set_title('denoise')
cb = fig.colorbar(im2)

In [12]:
from speechbrain.pretrained import SepformerSeparation as separator
# import torchaudio

model = separator.from_hparams(source="speechbrain/sepformer-whamr", savedir='pretrained_models/sepformer-whamr')

# for custom file, change path
est_sources = model.separate_file(path='speechbrain/sepformer-wsj02mix/test_mixture.wav') 

torchaudio.save("source1hat.wav", est_sources[:, :, 0].detach().cpu(), 8000)
torchaudio.save("source2hat.wav", est_sources[:, :, 1].detach().cpu(), 8000)

Downloading:   0%|          | 0.00/1.51k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/113M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.3k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/66.2k [00:00<?, ?B/s]

In [15]:
sep0 = sf.read('/home/work2020/yangwenhao/project/speechbrain/samples/audio_samples/test_mixture.wav')
IPython.display.Audio(sep0[0], rate=8000)

In [17]:
sep1 = sf.read('source1hat.wav')
IPython.display.Audio(sep1[0], rate=8000)

In [18]:
sep2 = sf.read('source2hat.wav')
IPython.display.Audio(sep2[0], rate=8000)

In [49]:
from speechbrain.pretrained import EncoderClassifier
classifier = EncoderClassifier.from_hparams(source="speechbrain/lang-id-commonlanguage_ecapa",
                                            savedir="pretrained_models/lang-id-commonlanguage_ecapa",
                                           run_opts={"device": "cuda",})
# Italian Example
out_prob, score, index, text_lab = classifier.classify_file('speechbrain/lang-id-commonlanguage_ecapa/example-it.wav')
print(text_lab)

# French Example
out_prob, score, index, text_lab = classifier.classify_file('speechbrain/lang-id-commonlanguage_ecapa/example-fr.wav')
print(text_lab)

['Italian']
['French']


In [25]:
chn = '/home/yangwenhao/storage/dataset/CN-Celeb/data/id00003/entertainment-01-001.wav'
out_prob, score, index, text_lab = classifier.classify_file(chn)
print(text_lab)
sep1 = sf.read(chn)
IPython.display.Audio(sep1[0], rate=16000)


['Chinese_China']


In [34]:
chn = '/home/yangwenhao/storage/dataset/CN-Celeb/data/id00003/interview-02-001.wav'

out_prob, score, index, text_lab = classifier.classify_file(chn)
print(score, index, text_lab)

sep1 = sf.read(chn)
IPython.display.Audio(sep1[0], rate=16000)

tensor([0.5768]) tensor([24]) ['English']


In [33]:
chn = '/home/yangwenhao/storage/dataset/CN-Celeb/data/id00004/entertainment-01-001.wav'
out_prob, score, index, text_lab = classifier.classify_file(chn)
print(score, index, text_lab)

sep1 = sf.read(chn)
IPython.display.Audio(sep1[0], rate=16000)

tensor([0.5775]) tensor([43]) ['Chinese_China']


In [32]:
chn = '/home/yangwenhao/storage/dataset/CN-Celeb/data/id00007/singing-01-001.wav'
out_prob, score, index, text_lab = classifier.classify_file(chn)
print(score, index, text_lab)

sep1 = sf.read(chn)
IPython.display.Audio(sep1[0], rate=16000)


tensor([-0.0167]) tensor([41]) ['Kabyle']


In [40]:
waveform, sr = torchaudio.load(upath, channels_first=False)
print(waveform.shape)

torch.Size([429477, 1])


In [54]:
# classifier = classifier.cuda()
with torch.no_grad():
    with open ('/home/work2020/yangwenhao/project/lstm_speaker_verification/data/cnceleb/dev/wav.scp', 'r') as f1, \
        open ('/home/work2020/yangwenhao/project/lstm_speaker_verification/data/cnceleb/dev/utt2lang', 'w') as f2:
        for l in tqdm(f1.readlines(), ncols=100):
            uid, upath = l.split()

            waveform, sr = torchaudio.load(upath, channels_first=False)
            if len(waveform) > 3*16000:
                waveform = waveform[:160000]

                batch = waveform.unsqueeze(0).squeeze(2).cuda()
                rel_length = torch.tensor([1.0])

                emb = classifier.encode_batch(batch, rel_length)
                out_prob = classifier.modules.classifier(emb).squeeze(1)
                score, index = torch.max(out_prob, dim=-1)
                text_lab = classifier.hparams.label_encoder.decode_torch(index)

        #         out_prob, score, index, text_lab = predicted_words[0]

                if score > 0.5:
                    f2.write(uid + ' '+ text_lab[0]+'\n')
        

100%|███████████████████████████████████████████████████████| 111259/111259 [46:47<00:00, 39.63it/s]
